# 4.1 Classifying movie reviews: A binary classification example

Two-class classification(Binary classification) 은 기계 학습에서 다루는 가장 일반적인 문제이다. 이 예제에서는 영화 리뷰를 긍정 또는 부정으로 분류하는 방법을 배운다.

## 4.1.1. The IMDB dataset

예제에서는 IMDB 데이터 세트를 사용해서 작업한다. IMDB 데이터 세트는 50,000개의 긍정/부정으로 구분되어 있는 리뷰 모음이다. 데이터는 25,000개의 학습 데이터와 25,000개의 검증 데이터로 나누어져 있다. 각 데이터 모둠은 50% 긍정과 50% 부정 리뷰로 구성되어 있다

**Loading the IMDB dataset**

In [1]:
from tensorflow.keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [2]:
# print(train_data[0])
# print(train_labels[0])
print(type(train_data))
print(train_data.shape)
print(len(train_data[0]), len(train_data[1]))

<class 'numpy.ndarray'>
(25000,)
218 189


In [3]:
max([max(sequence) for sequence in train_data])

9999

**Decoding review back to text**

In [4]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = " ".join(
    [reverse_word_index.get(i-3, "?") for i in train_data[0]]
)
print(decoded_review)

? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

## 4.1.2 Preparing the data

입력 데이터를 두 가지 방법으로 tensor로 변환해서 모델에 입력으로 사용한다.

 - 입력 데이터의 길이가 다르기 때문에 임의로 최대 크기로 지정한 후 최대 크기보다 작은 데이터는 최대 크기의 데이터로 맞춘다.
 - 입력 데이터를 (num_samples, max_length) 의 형태로 변환한다.
 - 변환한 입력 데이터를 네트워크(모델)에 입력으로 사용하는데, 이 경우 입력 데이터 값이 정수(예제의 경우)이기 때문에 정수 값을 그대로 사용하는 것이 아니라 데이터 변환 레이어(Embedding layer)를 네트워크의 앞에 두어 사용한다.
 
 - 다른 방법은 Multi-hot encode(one-hot embedding) 이다. 예제에서는 입력 데이터의 크기를 10,000으로 제한 했다. 따라서 10000차원의 벡터로 생각하고 실제 데이터가 있는 부분(단어가 있는 인덱스)은 1로 아닌 부분은 0으로 값이 채워진 벡터이다.
 - Dense layer에 입력으로 사용한다.

**Encoding the integer sequences via multi-hot encoding**

In [5]:
import numpy as np

def vectorize_sequences(sequences, dimention=10000):
    results = np.zeros((len(sequences), dimention))
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [6]:
print(x_train[0], len(x_train[0]))

[0. 1. 1. ... 0. 0. 0.] 10000


In [7]:
y_train = np.asarray(train_labels).astype("float32")
y_test =np.asarray(train_labels).astype("float32")

## 4.1.3 Building your model

입력 데이터로 벡터와 스칼라(label)로 구성하는 것이 가장 단순하게 문제를 정의하는 방법이다. 모델을 구성할 때 단순한 네트워크(몇 개의 레이러를 쌓아서 연결하는 구조)와 relu 활성함수를 사용하는 것이다.

네트워크를 구성할 때 결정해야 하는 중요한 요소 두 가지
- 얼마나 많은 레이어로 구성할 것인가?
- 각 레이어에 노드는 얼마나 많이 구성할 것인가?

이 예제에서는
- 두 개의 히든 레이어를 사용할 예정이다.
- 각 레이어는 16개의 노드(units)으로 구성한다.

**Model definition**

In [8]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

2022-01-15 15:08:38.738109: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-15 15:08:38.738781: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-15 15:08:38.743846: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-15 15:08:38.744523: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-15 15:08:38.745138: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

**TODO**
relu, sigmoid 활성화 함수에 대한 개형과 의미 설명 주가

In [9]:
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

## 4.1.4 Validating your approach

**Setting aside a validation set**

In [10]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [13]:
print(partial_x_train.shape)

(15000, 10000)


**Training  you model**

In [11]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

Epoch 1/20
28/30 [===========================>..] - ETA: 0s - loss: 0.5321 - accuracy: 0.7872

ValueError: in user code:

    File "/home/btb/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1330, in test_function  *
        return step_function(self, iterator)
    File "/home/btb/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1320, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/btb/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1313, in run_step  **
        outputs = model.test_step(data)
    File "/home/btb/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1267, in test_step
        y_pred = self(x, training=False)
    File "/home/btb/.local/lib/python3.9/site-packages/keras/engine/base_layer.py", line 1020, in __call__
        input_spec.assert_input_compatibility(self.input_spec, inputs, self.name)
    File "/home/btb/.local/lib/python3.9/site-packages/keras/engine/input_spec.py", line 199, in assert_input_compatibility
        raise ValueE